<font size=6>**接着mxnet_deeplearning file**<font/>

# 卷积神经网络
## 批量归一化
<font size=4>**主要让收敛的更快，对精度提升不大**</font><br/>
    
本节我们介绍批量归一化（batch normalization）层，它能让较深的神经网络的训练变得更加容易 [1]。在 “实战Kaggle比赛：预测房价” 一节里，我们对输入数据做了标准化处理：处理后的**任意一个特征**在数据集中所有样本上的均值为0、标准差为1。标准化处理输入数据使各个特征的分布相近：这往往更容易训练出有效的模型。

通常来说，数据标准化预处理对于浅层模型就足够有效了。随着模型训练的进行，当每层中参数更新时，靠近输出层的输出较难出现剧烈变化。但对深层神经网络来说，即使输入数据已做标准化，训练中模型参数的更新依然很容易造成靠近输出层输出的剧烈变化。这种计算数值的不稳定性通常令我们难以训练出有效的深度模型。

批量归一化的提出正是为了应对深度模型训练的挑战。在模型训练时，批量归一化利用小批量上的均值和标准差，不断调整神经网络中间输出，从而使整个神经网络在各层的中间输出的数值更稳定。批量归一化和下一节将要介绍的残差网络为训练和设计深度模型提供了两类重要思路。

### 批量归一化层
对全连接层和卷积层做批量归一化的方法稍有不同。下面我们将分别介绍这两种情况下的批量归一化。
#### 对全连接层做批量归一化
通常，我们将批量归一化层置于全连接层中的仿射变换和激活函数之间。设全连接层的输入为u，权重参数和偏差参数分别为W和b，激活函数为ϕ。设批量归一化的运算符为BN。那么，使用批量归一化的全连接层的输出为

$ϕ(BN(x))$,<br/>
其中批量归一化输入x由仿射变换

$x=Wu+b$<br/>
得到。考虑一个由m个样本组成的小批量，仿射变换的输出为一个新的小批量$B={x^{(1)},…,x^{(m)}}$。它们正是批量归一化层的输入。对于小批量B中任意样本$x^{(i)}∈R^d,1≤i≤m$，批量归一化层的输出同样是d维向量$y^{(i)}=BN(x^{(i)})$,

并由以下几步求得。首先，对小批量B求均值和方差：

$μ_B←\frac{1}{m} \sum_{i=1}^m x^{(i)}$,<br/>
$σ_B^2←\frac{1}{m} \sum_{i=1}^m (x^{(i)}-\mu_B)^2$,<br/>
其中的平方计算是按元素求平方。接下来，使用按元素开方和按元素除法对$x^{(i)}$标准化：

$\hat{x}^{(i)}←\frac{x^{(i)}−μ_B}{\sqrt{σ_B^2+ϵ}}$,<br/>
这里ϵ>0是一个很小的常数，保证分母大于0。在上面标准化的基础上，批量归一化层引入了两个可以学习的模型参数，拉伸（scale）参数 γ 和偏移（shift）参数 β。这两个参数和$x^{(i)}$形状相同，皆为d维向量。它们与$x^{(i)}$分别做按元素乘法（符号⊙）和加法计算：

$y^{(i)}←γ⊙\hat{x}^{(i)}+β$.<br/>
至此，我们得到了$x^{(i)}$的批量归一化的输出$y^{(i)}$。 值得注意的是，可学习的拉伸和偏移参数保留了不对$\hat{x}^{(i)}$做批量归一化的可能：此时只需学出$γ=\sqrt{σ_B^2+ϵ}$和$β=\mu_B$。我们可以对此这样理解：如果批量归一化无益，理论上，学出的模型可以不使用批量归一化。

#### 对卷积层做批量归一化
对卷积层来说，批量归一化发生在卷积计算之后、应用激活函数之前。如果卷积计算输出多个通道，我们需要对这些通道的输出分别做批量归一化，且每个通道都拥有独立的拉伸和偏移参数，并均为标量。设小批量中有 m 个样本。在单个通道上，假设卷积计算输出的高和宽分别为 p 和 q 。我们需要对该通道中 m×p×q 个元素同时做批量归一化。对这些元素做标准化计算时，我们使用相同的均值和方差，即该通道中 m×p×q 个元素的均值和方差。

#### 预测时的批量归一化
使用批量归一化训练时，我们可以将批量大小设得大一点，从而使批量内样本的均值和方差的计算都较为准确。将训练好的模型用于预测时，我们希望模型对于任意输入都有确定的输出。因此，单个样本的输出不应取决于批量归一化所需要的随机小批量中的均值和方差。一种常用的方法是通过移动平均估算整个训练数据集的样本均值和方差，并在预测时使用它们得到确定的输出。可见，和丢弃层一样，批量归一化层在训练模式和预测模式下的计算结果也是不一样的。

#### 从零开始实现

In [1]:
import d2lzh as d2l
from mxnet import autograd,nd,init,gluon
from mxnet.gluon import nn

In [6]:
def batch_norm(X,gamma,beta,moving_mean,moving_var,eps,momentum):
    if not autograd.is_training():
        # 如果是在预测模式下，直接使用传入的移动平均所得的均值和方差
        X_hat=(X-moving_mean)/nd.sqrt(moving_var+eps)
    else:
        assert len(X.shape) in (2,4)
        if len(X.shape)==2:
            #使用全连接情况
            mean=X.mean(axis=0)#求关于那个维度的均值就不写哪个维度。其余的都写,求各个特征的均值
            var=((X-mean)**2).mean(axis=0)
        else:
            # 使用二维卷积层的情况，计算通道维上（axis=1）的均值和方差。这里我们需要保持
            # X的形状以便后面可以做广播运算
            mean=X.mean(axis=(0,2,3),keepdims=True)#求关于那个维度的均值就不写哪个维度。其余的都写
            var=((X-mean)**2).mean(axis=(0,2,3),keepdims=True)
        X_hat=(X-mean)/nd.sqrt(var+eps)
        #更新移动的平均值和方差
        moving_mean=momentum*moving_mean+(1-momentum)*mean
        moving_var=momentum*moving_var+(1-momentum)*var
    Y=gamma*X_hat+beta
    return Y,moving_mean,moving_var

接下来，我们自定义一个BatchNorm层。它保存参与求梯度和迭代的拉伸参数gamma和偏移参数beta，同时也维护移动平均得到的均值和方差，以便能够在模型预测时被使用。BatchNorm实例所需指定的**num_features参数对于全连接层来说应为输出个数，对于卷积层来说则为输出通道数**。该实例所需指定的num_dims参数对于全连接层和卷积层来说分别为2和4。

In [9]:
class BatchNorm(nn.Block):
    def __init__(self,num_features,num_dims,**kwargs):
        super(BatchNorm,self).__init__(**kwargs)
        if num_dims==2:
            shape=(1,num_features)
        else:
            shape=(1,num_features,1,1)
        self.gamma=self.params.get('gamma',shape=shape,init=init.One())
        self.beta=self.params.get('beta',shape=shape,init=init.Zero())
        self.moving_mean=nd.zeros(shape)
        self.moving_var=nd.zeros(shape)
        
        
    def forward(self,X):
        # 如果X不在内存上，将moving_mean和moving_var复制到X所在显存上
        if self.moving_mean.context!=X.context:
            self.moving_mean=self.moving_mean.copyto(X.context)
            self.moving_var=self.moving_var.copyto(X.context)
         # 保存更新过的moving_mean和moving_var
        Y,self.moving_mean,self.moving_var=batch_norm(X,self.gamma.data(),self.beta.data(),self.moving_mean,self.moving_var,eps=1e-5,momentum=0)
        return Y

#### 使用批量归一化层的LeNet

In [10]:
net=nn.Sequential()
net.add(nn.Conv2D(6,kernel_size=5))
net.add(BatchNorm(6,num_dims=4))
net.add(nn.Activation('sigmoid'))
net.add(nn.MaxPool2D(pool_size=2,strides=2))
net.add(nn.Conv2D(16,kernel_size=5))
net.add(BatchNorm(16,num_dims=4))
net.add(nn.Activation('sigmoid'))
net.add(nn.MaxPool2D(pool_size=2,strides=2))
net.add(nn.Dense(120))
net.add(BatchNorm(120,num_dims=2))
net.add(nn.Activation('sigmoid'))
net.add(nn.Dense(84))
net.add(BatchNorm(84,num_dims=2))
net.add(nn.Activation('sigmoid'))
net.add(nn.Dense(10))

In [13]:
lr,num_epochs,batch_size,ctx=1.0,5,256,d2l.try_gpu()
net.initialize(ctx=ctx,init=init.Xavier(),force_reinit=True)
trainer=gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':lr})
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size=batch_size)
d2l.train_ch5(net,train_iter,test_iter,batch_size,trainer,ctx,num_epochs)

training on gpu(0)
epoch 1, loss 0.6452, train acc 0.770, test acc 0.857, time 9.0 sec
epoch 2, loss 0.3974, train acc 0.855, test acc 0.865, time 8.8 sec
epoch 3, loss 0.3438, train acc 0.875, test acc 0.873, time 8.7 sec
epoch 4, loss 0.3183, train acc 0.884, test acc 0.881, time 8.9 sec
epoch 5, loss 0.3008, train acc 0.891, test acc 0.887, time 8.8 sec


In [14]:
net[1].gamma.data().reshape((-1,)),net[1].beta.data().reshape((-1,))

(
 [1.825248  1.9441062 0.9561216 1.7321875 2.191279  1.5300164]
 <NDArray 6 @gpu(0)>, 
 [-0.16317885  0.24874806 -0.77239984 -0.20372763 -2.2464998  -0.24971762]
 <NDArray 6 @gpu(0)>)

#### 简洁实现

In [15]:
net=nn.Sequential()
net.add(nn.Conv2D(6,kernel_size=5))
net.add(nn.BatchNorm())
net.add(nn.Activation('sigmoid'))
net.add(nn.MaxPool2D(pool_size=2,strides=2))
net.add(nn.Conv2D(16,kernel_size=5))
net.add(nn.BatchNorm())
net.add(nn.Activation('sigmoid'))
net.add(nn.MaxPool2D(pool_size=2,strides=2))
net.add(nn.Dense(120))
net.add(nn.BatchNorm())
net.add(nn.Activation('sigmoid'))
net.add(nn.Dense(84))
net.add(nn.BatchNorm())
net.add(nn.Activation('sigmoid'))
net.add(nn.Dense(10))

In [16]:
net.initialize(ctx=ctx,init=init.Xavier())
trainer=gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':lr})
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size=batch_size)
d2l.train_ch5(net,train_iter,test_iter,batch_size,trainer,ctx,num_epochs)

training on gpu(0)
epoch 1, loss 0.6641, train acc 0.765, test acc 0.805, time 5.6 sec
epoch 2, loss 0.4036, train acc 0.854, test acc 0.846, time 5.4 sec
epoch 3, loss 0.3521, train acc 0.873, test acc 0.866, time 5.7 sec
epoch 4, loss 0.3241, train acc 0.883, test acc 0.872, time 5.5 sec
epoch 5, loss 0.3067, train acc 0.888, test acc 0.889, time 5.4 sec


In [17]:
lr=1.5
net.initialize(ctx=ctx,init=init.Xavier(),force_reinit=True)
trainer=gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':lr})
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size=batch_size)
d2l.train_ch5(net,train_iter,test_iter,batch_size,trainer,ctx,num_epochs)

training on gpu(0)
epoch 1, loss 0.7263, train acc 0.757, test acc 0.836, time 5.5 sec
epoch 2, loss 0.4002, train acc 0.856, test acc 0.870, time 5.4 sec
epoch 3, loss 0.3496, train acc 0.873, test acc 0.827, time 5.4 sec
epoch 4, loss 0.3219, train acc 0.883, test acc 0.873, time 5.5 sec
epoch 5, loss 0.3009, train acc 0.891, test acc 0.874, time 5.4 sec


* 尝试一下不学习拉伸参数gamma和偏移参数beta（构造的时候加入参数grad_req='null'来避免计算梯度），观察并分析结果。

In [18]:
class BatchNorm(nn.Block):
    def __init__(self,num_features,num_dims,**kwargs):
        super(BatchNorm,self).__init__(**kwargs)
        if num_dims==2:
            shape=(1,num_features)
        else:
            shape=(1,num_features,1,1)
        self.gamma=self.params.get('gamma',shape=shape,init=init.One())
        self.gamma.grad_req='null'  #对gamma不求梯度
        self.beta=self.params.get('beta',shape=shape,init=init.Zero())
        self.beta.grad_req='null'
        self.moving_mean=nd.zeros(shape)
        self.moving_var=nd.zeros(shape)
        
        
    def forward(self,X):
        # 如果X不在内存上，将moving_mean和moving_var复制到X所在显存上
        if self.moving_mean.context!=X.context:
            self.moving_mean=self.moving_mean.copyto(X.context)
            self.moving_var=self.moving_var.copyto(X.context)
         # 保存更新过的moving_mean和moving_var
        Y,self.moving_mean,self.moving_var=batch_norm(X,self.gamma.data(),self.beta.data(),self.moving_mean,self.moving_var,eps=1e-5,momentum=0)
        return Y

In [19]:
net=nn.Sequential()
net.add(nn.Conv2D(6,kernel_size=5))
net.add(BatchNorm(6,num_dims=4))
net.add(nn.Activation('sigmoid'))
net.add(nn.MaxPool2D(pool_size=2,strides=2))
net.add(nn.Conv2D(16,kernel_size=5))
net.add(BatchNorm(16,num_dims=4))
net.add(nn.Activation('sigmoid'))
net.add(nn.MaxPool2D(pool_size=2,strides=2))
net.add(nn.Dense(120))
net.add(BatchNorm(120,num_dims=2))
net.add(nn.Activation('sigmoid'))
net.add(nn.Dense(84))
net.add(BatchNorm(84,num_dims=2))
net.add(nn.Activation('sigmoid'))
net.add(nn.Dense(10))

lr,num_epochs,batch_size,ctx=1.0,5,256,d2l.try_gpu()
net.initialize(ctx=ctx,init=init.Xavier(),force_reinit=True)
trainer=gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':lr})
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size=batch_size)
d2l.train_ch5(net,train_iter,test_iter,batch_size,trainer,ctx,num_epochs)

training on gpu(0)
epoch 1, loss 0.6544, train acc 0.769, test acc 0.849, time 8.0 sec
epoch 2, loss 0.4202, train acc 0.849, test acc 0.871, time 8.0 sec
epoch 3, loss 0.3697, train acc 0.868, test acc 0.877, time 8.0 sec
epoch 4, loss 0.3416, train acc 0.880, test acc 0.884, time 8.0 sec
epoch 5, loss 0.3205, train acc 0.885, test acc 0.888, time 8.1 sec


In [20]:
net[1].gamma.data().reshape((-1,)),net[1].beta.data().reshape((-1,))

(
 [1. 1. 1. 1. 1. 1.]
 <NDArray 6 @gpu(0)>, 
 [0. 0. 0. 0. 0. 0.]
 <NDArray 6 @gpu(0)>)

## 使用重复元素的网络(VGG)
AlexNet在LeNet的基础上增加了3个卷积层。但AlexNet作者对它们的卷积窗口、输出通道数和构造顺序均做了大量的调整。虽然AlexNet指明了深度卷积神经网络可以取得出色的结果，但并没有提供简单的规则以指导后来的研究者如何设计新的网络。我们将在本章的后续几节里介绍几种不同的深度网络设计思路。

本节介绍VGG，它的名字来源于论文作者所在的实验室Visual Geometry Group [1]。VGG提出了可以通过重复使用简单的基础块来构建深度模型的思路。
### VGG块
VGG块的组成规律是：连续使用数个相同的填充为1、窗口形状为 3×3 的卷积层后接上一个步幅为2、窗口形状为 2×2 的最大池化层。卷积层保持输入的高和宽不变，而池化层则对其减半。我们使用vgg_block函数来实现这个基础的VGG块，它可以指定卷积层的数量num_convs和输出通道数num_channels。

In [1]:
import d2lzh as d2l
from mxnet import init,gluon,nd
from mxnet.gluon import nn

def vgg_block(num_convs,num_channels):
    blk=nn.Sequential()
    for _ in range(num_convs):
        blk.add(nn.Conv2D(num_channels,kernel_size=3,padding=1,activation='relu'))
    blk.add(nn.MaxPool2D(pool_size=2,strides=2))
    return blk


## VGG网络
与AlexNet和LeNet一样，VGG网络由卷积层模块后接全连接层模块构成。卷积层模块串联数个vgg_block，其超参数由变量conv_arch定义。该变量指定了每个VGG块里卷积层个数和输出通道数。全连接模块则跟AlexNet中的一样。

现在我们构造一个VGG网络。它有5个卷积块，前2块使用单卷积层，而后3块使用双卷积层。第一块的输出通道是64，之后每次对输出通道数翻倍，直到变为512。因为这个网络使用了8个卷积层和3个全连接层，所以经常被称为VGG-11。

![VGGNet不同配置](https://img-blog.csdn.net/20170715114221637?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvdTAxNDI4MTM5Mg==/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/SouthEast)

In [2]:
conv_arch=((1,64),(1,128),(2,256),(2,512),(2,512))

In [3]:
def vgg(conv_arch):
    net=nn.Sequential()
    #卷积部分
    for (num_convs,num_channels) in conv_arch:
        net.add(vgg_block(num_convs,num_channels))
    #全连接层
    net.add(nn.Dense(4096,activation='relu'))
    net.add(nn.Dropout(0.5))
    net.add(nn.Dense(4096,activation='relu'))
    net.add(nn.Dropout(0.5))
    net.add(nn.Dense(10))
    return net

net=vgg(conv_arch=conv_arch)

In [4]:
net.initialize(force_reinit=True)
X=nd.random.uniform(shape=(1,1,224,224))
for blk in net:
    X=blk(X)
    print(blk.name,'out shape:',X.shape)

sequential1 out shape: (1, 64, 112, 112)
sequential2 out shape: (1, 128, 56, 56)
sequential3 out shape: (1, 256, 28, 28)
sequential4 out shape: (1, 512, 14, 14)
sequential5 out shape: (1, 512, 7, 7)
dense0 out shape: (1, 4096)
dropout0 out shape: (1, 4096)
dense1 out shape: (1, 4096)
dropout1 out shape: (1, 4096)
dense2 out shape: (1, 10)


可以看到，每次我们将输入的高和宽减半，直到最终高和宽变成4后传入全连接层。与此同时，输出通道数每次翻倍，直到变成512。因为每个卷积层的窗口大小一样，所以每层的模型参数尺寸和计算复杂度与输入高、输入宽、输入通道数和输出通道数的乘积成正比。VGG这种高和宽减半以及通道翻倍的设计使得多数卷积层都有相同的模型参数尺寸和计算复杂度。

### 获取数据和训练模型
因为VGG-11计算上比AlexNet更加复杂，出于测试的目的我们构造一个通道数更小，或者说更窄的网络在Fashion-MNIST数据集上进行训练

In [5]:
ratio=8
small_conv_arch=[(pair[0],pair[1]//ratio) for pair in conv_arch]
small_conv_arch

[(1, 8), (1, 16), (2, 32), (2, 64), (2, 64)]

In [6]:
lr,num_epochs,batch_size,ctx=0.05,5,128,d2l.try_gpu()
net=vgg(small_conv_arch)
net.initialize(ctx=ctx,init=init.Xavier())
trainer=gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':lr})
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size,resize=224)
d2l.train_ch5(net,train_iter,test_iter,batch_size,trainer,ctx,num_epochs)

training on gpu(0)
epoch 1, loss 0.8286, train acc 0.701, test acc 0.841, time 456.0 sec
epoch 2, loss 0.4055, train acc 0.853, test acc 0.882, time 450.3 sec


KeyboardInterrupt: 

## 网络中的网络（NiN）
前几节介绍的LeNet、AlexNet和VGG在设计上的共同之处是：先以由卷积层构成的模块充分抽取空间特征，再以由全连接层构成的模块来输出分类结果。其中，AlexNet和VGG对LeNet的改进主要在于如何对这两个模块加宽（增加通道数）和加深。本节我们介绍网络中的网络（NiN）[1]。它提出了另外一个思路，即串联多个由卷积层和“全连接”层构成的小网络来构建一个深层网络。

![](https://images2017.cnblogs.com/blog/606386/201710/606386-20171011104028715-922103502.png)

### NiN块
我们知道，卷积层的输入和输出通常是四维数组（样本，通道，高，宽），而全连接层的输入和输出则通常是二维数组（样本，特征）。如果想在全连接层后再接上卷积层，则需要将全连接层的输出变换为四维。回忆在“多输入通道和多输出通道”一节里介绍的 1×1 卷积层。它可以看成全连接层，其中空间维度（高和宽）上的每个元素相当于样本，通道相当于特征。因此，NiN使用 1×1 卷积层来替代全连接层，从而使空间信息能够自然传递到后面的层中去。图5.7对比了NiN同AlexNet和VGG等网络在结构上的主要区别。左图是AlexNet和VGG的网络结构局部，右图是NiN的网络结构局部。

![](http://zh.d2l.ai/_images/nin.svg)

NiN块是NiN中的基础块。它由一个卷积层加两个充当全连接层的 1×1 卷积层串联而成。其中第一个卷积层的超参数可以自行设置，而第二和第三个卷积层的超参数一般是固定的。

In [1]:
import d2lzh as d2l
from mxnet import gluon,init,nd
from mxnet.gluon import nn

In [2]:
def nin_block(num_channels,kernel_size,strides,padding):
    blk=nn.Sequential()
    blk.add(nn.Conv2D(num_channels,kernel_size=kernel_size,strides=strides,padding=padding,activation='relu'))
    blk.add(nn.Conv2D(num_channels,kernel_size=1,activation='relu'))
    blk.add(nn.Conv2D(num_channels,kernel_size=1,activation='relu'))
    return blk

### NiN模型
NiN是在AlexNet问世不久后提出的。它们的卷积层设定有类似之处。NiN使用卷积窗口形状分别为 11×11 、 5×5 和 3×3 的卷积层，相应的输出通道数也与AlexNet中的一致。每个NiN块后接一个步幅为2、窗口形状为 3×3 的最大池化层。

除使用NiN块以外，NiN还有一个设计与AlexNet显著不同：NiN去掉了AlexNet最后的3个全连接层，取而代之地，NiN使用了输出通道数等于标签类别数的NiN块，然后使用全局平均池化层对每个通道中所有元素求平均并直接用于分类。这里的全局平均池化层即窗口形状等于输入空间维形状的平均池化层。NiN的这个设计的好处是可以显著减小模型参数尺寸，从而缓解过拟合。然而，该设计有时会造成获得有效模型的训练时间的增加。

In [3]:
net=nn.Sequential()
net.add(nin_block(96,kernel_size=11,strides=4,padding=0))
net.add(nn.MaxPool2D(pool_size=3,strides=2))
net.add(nin_block(256,kernel_size=5,strides=1,padding=2))
net.add(nn.MaxPool2D(pool_size=3,strides=2))
net.add(nin_block(384,kernel_size=3,strides=1,padding=1))
net.add(nn.MaxPool2D(pool_size=3,strides=2))
net.add(nn.Dropout(0.5))
# 类别数为10
net.add(nin_block(10,kernel_size=3,strides=1,padding=1))
# 全局平均池化层将窗口形状自动设置成输入的高和宽
net.add(nn.GlobalAvgPool2D())
net.add(nn.Flatten())

In [4]:
X=nd.random.uniform(shape=(1,1,224,224))
net.initialize()
for layer in net:
    X=layer(X)
    print(layer.name,'output shape:',X.shape)

sequential1 output shape: (1, 96, 54, 54)
pool0 output shape: (1, 96, 26, 26)
sequential2 output shape: (1, 256, 26, 26)
pool1 output shape: (1, 256, 12, 12)
sequential3 output shape: (1, 384, 12, 12)
pool2 output shape: (1, 384, 5, 5)
dropout0 output shape: (1, 384, 5, 5)
sequential4 output shape: (1, 10, 5, 5)
pool3 output shape: (1, 10, 1, 1)
flatten0 output shape: (1, 10)


### 获取数据和训练模型

In [5]:
lr,num_epochs,batch_size,ctx=0.1,1,128,d2l.try_gpu()
net.initialize(init=init.Xavier(),force_reinit=True,ctx=ctx)
trainer=gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':lr})
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size,resize=224)
d2l.train_ch5(net,train_iter,test_iter,batch_size,trainer,ctx,num_epochs)

training on gpu(0)
epoch 1, loss 2.2009, train acc 0.161, test acc 0.340, time 234.8 sec


## 含并行连接的网络（GoogleNet）
在2014年的ImageNet图像识别挑战赛中，一个名叫GoogLeNet的网络结构大放异彩 [1]。它虽然在名字上向LeNet致敬，但在网络结构上已经很难看到LeNet的影子。GoogLeNet吸收了NiN中网络串联网络的思想，并在此基础上做了很大改进。在随后的几年里，研究人员对GoogLeNet进行了数次改进，本节将介绍这个模型系列的第一个版本。

### Inception块
GoogLeNet中的基础卷积块叫作Inception块，得名于同名电影《盗梦空间》（Inception）。与上一节介绍的NiN块相比，这个基础块在结构上更加复杂，如图5.8所示。

![](http://zh.d2l.ai/_images/inception.svg)
由图5.8可以看出，Inception块里有4条并行的线路。前3条线路使用窗口大小分别是 1×1 、 3×3 和 5×5 的卷积层来抽取不同空间尺寸下的信息，其中中间2个线路会对输入先做 1×1 卷积来减少输入通道数，以降低模型复杂度。第四条线路则使用 3×3 最大池化层，后接 1×1 卷积层来改变通道数。4条线路都使用了合适的填充来使输入与输出的高和宽一致。最后我们将每条线路的输出在通道维上连结，并输入接下来的层中去。

Inception块中可以自定义的超参数是每个层的输出通道数，我们以此来控制模型复杂度。

In [6]:
import d2lzh as d2l
from mxnet import nd,init,gluon
from mxnet.gluon import nn

In [17]:
class Inception(nn.Block):
    #c1-c4为每条线路中的层的输出通道数
    def __init__(self,c1,c2,c3,c4,**kwargs):
        super(Inception,self).__init__(**kwargs)
        
        #线路1，1*1卷积层
        self.p1_1=nn.Conv2D(c1,kernel_size=1,activation='relu')
        
        #线路2，1*1卷积和3*3卷积
        self.p2_1=nn.Conv2D(c2[0],kernel_size=1,activation='relu')
        self.p2_2=nn.Conv2D(c2[1],kernel_size=3,padding=1,strides=1,activation='relu')
        
        #线路3，1*1卷积和5*5卷积
        self.p3_1=nn.Conv2D(c3[0],kernel_size=1,activation='relu')
        self.p3_2=nn.Conv2D(c3[1],kernel_size=5,padding=2,strides=1,activation='relu')
        
        #线路4，3*3 maxpool和1*1卷积
        self.p4_1=nn.MaxPool2D(pool_size=3,padding=1,strides=1)
        self.p4_2=nn.Conv2D(c4,kernel_size=1,activation='relu')
        
    def forward(self,X):
        p1=self.p1_1(X)
        p2=self.p2_2(self.p2_1(X))
        p3=self.p3_2(self.p3_1(X))
        p4=self.p4_2(self.p4_1(X))
        return nd.concat(p1,p2,p3,p4,dim=1)#在通道为上连接
        

### GoogLeNet模型
GoogLeNet跟VGG一样，在主体卷积部分中使用5个模块（block），每个模块之间使用步幅为2的 3×3 最大池化层来减小输出高宽。第一模块使用一个64通道的 7×7 卷积层。
采用inception的GoogLeNet如下：
![](https://images2015.cnblogs.com/blog/822124/201609/822124-20160902160824621-1952207446.png)

In [18]:
b1=nn.Sequential()
b1.add(nn.Conv2D(64,kernel_size=7,strides=2,padding=3,activation='relu'))
b1.add(nn.MaxPool2D(pool_size=3,strides=2,padding=1))

第二模块使用2个卷积层：首先是64通道的 1×1 卷积层，然后是将通道增大3倍的 3×3 卷积层。它对应Inception块中的第二条线路。

In [19]:
b2=nn.Sequential()
b2.add(nn.Conv2D(64,kernel_size=1))
b2.add(nn.Conv2D(192,kernel_size=3,padding=1)) #未指定激活函数则是线性激活函数f(x)=x，即没有激活函数，直接输出
b2.add(nn.MaxPool2D(pool_size=3,strides=2,padding=1))

第三模块串联2个完整的Inception块。第一个Inception块的输出通道数为64+128+32+32=256，其中4条线路的输出通道数比例为64:128:32:32=2:4:1:1。其中第二、第三条线路先分别将输入通道数减小至96/192=1/2和16/192=1/12后，再接上第二层卷积层。第二个Inception块输出通道数增至128+192+96+64=480，每条线路的输出通道数之比为128:192:96:64=4:6:3:2。其中第二、第三条线路先分别将输入通道数减小至128/256=1/2和32/256=1/8。

In [20]:
b3=nn.Sequential()
b3.add(Inception(64,[96,128],[16,32],32))
b3.add(Inception(128,[128,192],[32,96],64))
b3.add(nn.MaxPool2D(pool_size=3,strides=2,padding=1))

第四模块更加复杂。它串联了5个Inception块，其输出通道数分别是 192+208+48+64=512 、 160+224+64+64=512 、 128+256+64+64=512 、 112+288+64+64=528 和 256+320+128+128=832 。这些线路的通道数分配和第三模块中的类似，首先含 3×3 卷积层的第二条线路输出最多通道，其次是仅含 1×1 卷积层的第一条线路，之后是含 5×5 卷积层的第三条线路和含 3×3 最大池化层的第四条线路。其中第二、第三条线路都会先按比例减小通道数。这些比例在各个Inception块中都略有不同。

In [21]:
b4=nn.Sequential()
b4.add(Inception(192,[96,108],[16,48],64))
b4.add(Inception(160,[112,224],[24,64],64))
b4.add(Inception(128,[128,256],[34,64],64))
b4.add(Inception(112,[144,288],[32,64],64))
b4.add(Inception(256,[160,320],[32,128],128))
b4.add(nn.MaxPool2D(pool_size=3,strides=2,padding=1))

第五模块有输出通道数为 256+320+128+128=832 和 384+384+128+128=1024 的两个Inception块。其中每条线路的通道数的分配思路和第三、第四模块中的一致，只是在具体数值上有所不同。需要注意的是，第五模块的后面紧跟输出层，该模块同NiN一样使用全局平均池化层来将每个通道的高和宽变成1。最后我们将输出变成二维数组后接上一个输出个数为标签类别数的全连接层。

In [22]:
b5=nn.Sequential()
b5.add(Inception(256,[160,320],[32,128],128))
b5.add(Inception(384,[192,384],[48,128],128))
b5.add(nn.GlobalAvgPool2D())

In [23]:
net=nn.Sequential()
net.add(b1)
net.add(b2)
net.add(b3)
net.add(b4)
net.add(b5)
net.add(nn.Dense(10))

GoogLeNet模型的计算复杂，而且不如VGG那样便于修改通道数。本节里我们将输入的高和宽从224降到96来简化计算。下面演示各个模块之间的输出的形状变化。

In [24]:
X=nd.random.uniform(shape=(1,1,96,96))
net.initialize()
for layer in net:
    X=layer(X)
    print(layer.name,'output shape:',X.shape)

sequential11 output shape: (1, 64, 24, 24)
sequential12 output shape: (1, 192, 12, 12)
sequential13 output shape: (1, 480, 6, 6)
sequential14 output shape: (1, 832, 3, 3)
sequential15 output shape: (1, 1024, 1, 1)
dense1 output shape: (1, 10)


### 获取数据和训练模型

In [25]:
lr,num_epochs,batch_size,ctx=0.1,1,256,d2l.try_gpu()
net.initialize(ctx=ctx,force_reinit=True,init=init.Xavier())
trainer=gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':lr})

train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size,resize=96)
d2l.train_ch5(net,train_iter,test_iter,batch_size,trainer,ctx,num_epochs)

training on gpu(0)
epoch 1, loss 2.2432, train acc 0.200, test acc 0.483, time 173.3 sec
